In [1]:
import pandas as pd

In [9]:
df1 = pd.read_csv("/Users/raghavraahul/Downloads/Fake.csv")
df2 = pd.read_csv("/Users/raghavraahul/Downloads/True.csv")


In [10]:
df1["Label"] = [1]*df1.shape[0]
df2['Label'] = [0]*df2.shape[0]
df = pd.concat([df1, df2], axis = 0)
#print(df.shape)
df.head()

,title,text,subject,date,Label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [11]:
df = df.drop(["subject", "date", "title"], axis = 1)
df.head()

,text,Label
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1


In [12]:
df.shape

(44898, 2)

In [13]:
df.Label.value_counts()

1    23481
0    21417
Name: Label, dtype: int64

In [14]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [15]:
#create new column to store vector for the textnegative
df["vector"] = df["text"].apply(lambda x: nlp(x).vector)

In [16]:
df.head()

,text,Label,vector
0,Donald Trump just couldn t wish all Americans ...,1,"[-1.6387042, -0.027837504, -1.6619873, -0.2017..."
1,House Intelligence Committee Chairman Devin Nu...,1,"[-1.9558355, 0.5587237, -2.036446, 0.40390933,..."
2,"On Friday, it was revealed that former Milwauk...",1,"[-1.9058484, -0.04223674, -1.7389339, -0.02855..."
3,"On Christmas day, Donald Trump announced that ...",1,"[-1.6265213, 0.14619705, -0.5877981, 0.0992525..."
4,Pope Francis used his annual Christmas Day mes...,1,"[-2.1327376, 1.1017615, -2.4795847, -0.0065187..."


In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.vector.values, df.Label, test_size = 0.2, random_state = 2023)

In [18]:
#need to convert X_train and X_test into 2d array for classification model
import numpy as np

X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [19]:
from sklearn.naive_bayes import MultinomialNB
#vectors have negative values which are not acceptable for Multinomial NB

#use minmaxscaler to address
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler_train_embed = scaler.fit_transform(X_train_2d)
scaler_test_embed = scaler.transform(X_test_2d)

clf = MultinomialNB()

clf.fit(scaler_train_embed, y_train)

MultinomialNB()

In [21]:
y_pred = clf.predict(scaler_test_embed)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86      4426
           1       0.88      0.84      0.86      4554

    accuracy                           0.86      8980
   macro avg       0.86      0.86      0.86      8980
weighted avg       0.86      0.86      0.86      8980



In [24]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean')
clf.fit(X_train_2d, y_train)
y_pred = clf.predict(X_test_2d)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      4426
           1       0.97      0.97      0.97      4554

    accuracy                           0.97      8980
   macro avg       0.97      0.97      0.97      8980
weighted avg       0.97      0.97      0.97      8980



In [ ]:
#knn really works well with a dense representation compared to sparse representation